# 장애인콜시스템
- [서울시 열린데이터 광장](https://data.seoul.go.kr/search/newSearch.jsp?query=%EC%9E%A5%EC%95%A0%EC%9D%B8%EC%BD%9C%ED%83%9D%EC%8B%9C)

## STEP 01 : 장애인 콜시스템 데이터 수집

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import os

In [6]:
file_path = r"D:\Workspace\Python_alone\pjt"

In [7]:
df = pd.DataFrame()
for date in tqdm_notebook(GetDate('2017-05-01', '2017-08-31')): # 날짜 수정(각 월에 맞게) !!! 
    df = pd.concat([df, load_cst_data(date)], axis=0)
    
df.to_csv(file_path+'\2017_2.csv', encoding='euc-kr', index=False) # 파일 이름 수정( 01~12 )!!! 파일 경로 수정!!! 

- file_path 경로 수정 필요

- 파일명 : cst_19_0103 : call system data (장애인콜시스템) + 데이터연도 + 수집월

In [3]:
def load_cst_data(date):
    URL = f'http://openapi.seoul.go.kr:8088/676a7359436c696d33394451454277/xml/disabledCalltaxi/1/100/{date}'
    res = requests.get(URL)
    html = res.text
    soup = BeautifulSoup(html, 'lxml')
    
    no = []
    cartype = []
    receipttime = []
    settime = []
    ridetime = []
    startpos1 = []
    startpos2 = []
    endpos1 = []
    endpos2 = []
    
    for items in soup.findAll('item'):
        no.append(items.find('no').text)
        cartype.append(items.find('cartype').text.replace(' ', ''))
        receipttime.append(items.find('receipttime').text)
        settime.append(items.find('settime').text)
        ridetime.append(items.find('ridetime').text)
        startpos1.append(items.find('startpos1').text)
        startpos2.append(items.find('startpos2').text)
        endpos1.append(items.find('endpos1').text)
        endpos2.append(items.find('endpos2').text)
        
    dfs = pd.DataFrame({'car_no':no,
                   'car_type':cartype,
                   'exp_time':receipttime,
                   'set_time':settime,
                   'ride_time':ridetime,
                   'dept_gu':startpos1,
                   'dept_dong':startpos2,
                   'dest_gu':endpos1,
                   'dest_dong':endpos2})
    
    return dfs
        

def GetDate(start, end):
    dateset = pd.date_range(start=start, end=end, freq='D')
    dates = []
    for date in dateset:
        dates.append(str(date)[:10])
    return dates

### **월단위 파일 합치기**

In [26]:
import os
import pandas as pd 

rdfs = [] # 빈 리스트 생성 
for file in os.listdir(file_path):
    if file.startswith("19_cst"): #시작이 "19_cst"로 시작하는 경우
        rdfs.append(pd.read_csv(file_path+"/"+file, encoding='euc-kr', index_col=None)) # 각 리스트에 데이터프레임 저장 

In [27]:
rdfs[1].head()

,car_no,car_type,exp_time,set_time,ride_time,dept_gu,dept_dong,dest_gu,dest_dong,ride_exp,ride_set,set_exp,month,day,hour,weekday
0,5815,중형승합,2019-01-01 00:02:00,2019-01-01 03:56:24,2019-01-01 04:15:34,종로구,삼청동,성북구,월곡1동,254,19,234,1,1,4,Tuesday
1,5815,중형승합,2019-01-01 00:13:00,2019-01-01 03:17:56,2019-01-01 03:36:11,성북구,보문동,강북구,삼각산동,203,18,185,1,1,3,Tuesday
2,1313,중형승합,2019-01-01 00:15:00,2019-01-01 03:03:30,2019-01-01 03:23:48,강북구,인수동,노원구,하계2동,189,20,168,1,1,3,Tuesday
3,5764,승합,2019-01-01 00:17:00,2019-01-01 03:55:50,2019-01-01 04:16:51,중구,회현동,부천시소사구,송내1동,240,21,219,1,1,4,Tuesday
4,1601,중형승합,2019-01-01 00:21:00,2019-01-01 01:36:00,2019-01-01 02:13:11,은평구,구산동,은평구,역촌동,112,37,75,1,1,2,Tuesday


In [28]:
dfs = pd.DataFrame() # 빈 데이터프레임 생성 
for rdf in rdfs:
    dfs = pd.concat([dfs, rdf], axis=0) # 리스트에 있는 데이터를 하나씩 추가 

In [29]:
len(dfs)

9240

In [33]:
dfs.to_csv(file_path+'/cst_19_raw.csv', index=None, encoding='euc-kr') # 연간데이터 파일 저장 